# Answering Questions for the Chinook Record Store

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name - Genre

- Regal - Hip-Hop

- Red Tone - Punk

- Meteor and the Girls - Pop

- Slim Jim Bites - Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Overview of the Data

Query the database to get a list of all tables and views in our database:

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting Albums to Purchase

Write a query that returns each genre, with the number of tracks sold in the USA:
- in absolute numbers
- in percentages

In [3]:
%%sql
SELECT il.* FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
INNER JOIN customer c on c.customer_id = i.customer_id
WHERE c.country = 'USA'
LIMIT 10;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


In [4]:
%%sql
WITH usa_sold AS(SELECT il.* FROM invoice_line il
                 INNER JOIN invoice i ON i.invoice_id = il.invoice_id
                 INNER JOIN customer c on c.customer_id = i.customer_id
                 WHERE c.country = 'USA'
                )
SELECT g.name genre,
       count(us.invoice_line_id) tracks_sold
FROM usa_sold us
INNER JOIN track t ON t.track_id = us.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2;

Done.


genre,tracks_sold
TV Shows,1
Soundtrack,2
Heavy Metal,3
Classical,4
Electronica/Dance,5
Reggae,6
Easy Listening,13
Jazz,14
Hip Hop/Rap,20
Latin,22


In [5]:
%%sql
WITH usa_sold AS(SELECT il.* FROM invoice_line il
                 INNER JOIN invoice i ON i.invoice_id = il.invoice_id
                 INNER JOIN customer c on c.customer_id = i.customer_id
                 WHERE c.country = 'USA'
                )
SELECT g.name genre,
       count(us.invoice_line_id) tracks_sold,
       cast(count(us.invoice_line_id) AS FLOAT) / (
       SELECT COUNT(*) from usa_sold
       ) percentage_sold
FROM usa_sold us
INNER JOIN track t ON t.track_id = us.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
 

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the table above and the four options mentioned at the beginning, the albums to choose would be from should be:

- Red Tone - Punk
- Slim Jim Bites - Blues
- Meteor and the Girls - Pop

In [6]:
%%sql
WITH usa_sold AS(SELECT il.* FROM invoice_line il
                 INNER JOIN invoice i ON i.invoice_id = il.invoice_id
                 INNER JOIN customer c on c.customer_id = i.customer_id
                 WHERE c.country = 'USA'
                )
SELECT DISTINCT t.composer artist_name,
       g.name genre,
       count(us.invoice_line_id) tracks_sold,
       cast(count(us.invoice_line_id) AS FLOAT) / (
       SELECT COUNT(*) from usa_sold
       ) percentage_sold
FROM usa_sold us
INNER JOIN track t ON t.track_id = us.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
WHERE g.name = 'Rock'
GROUP BY 1
ORDER BY 3 DESC
LIMIT 5;


Done.


artist_name,genre,tracks_sold,percentage_sold
None,Rock,70,0.06660323501427212
Jagger/Richards,Rock,30,0.028544243577545196
Kurt Cobain,Rock,28,0.02664129400570885
Jimi Hendrix,Rock,25,0.023786869647954328
"Robby Krieger, Ray Manzarek, John Densmore, Jim Morrison",Rock,23,0.021883920076117985


Rock, Alternative & Punk, and Metal have the most tracks sold. Adding more artists along the lines of The Rolling Stones and Nirvana would be beneficial after adding the three albums from above.

# Analyzing Employee Sales Performance

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [7]:
%%sql
SELECT SUM(i.total) total_sales,
       c.support_rep_id
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
GROUP BY 2

Done.


total_sales,support_rep_id
1731.510000000004,3
1584.0000000000032,4
1393.9200000000028,5


There is data on three sales agents.

In [8]:
%%sql
SELECT e.first_name ||' '|| e.last_name employee_name,
       e.title
  FROM employee e

Done.


employee_name,title
Andrew Adams,General Manager
Nancy Edwards,Sales Manager
Jane Peacock,Sales Support Agent
Margaret Park,Sales Support Agent
Steve Johnson,Sales Support Agent
Michael Mitchell,IT Manager
Robert King,IT Staff
Laura Callahan,IT Staff


Confirming the three sales agents.

In [9]:
%%sql
WITH total_sales_report AS
    (
     SELECT SUM(i.total) total,
            c.support_rep_id
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
     GROUP BY 2
    )
SELECT e.first_name ||' '|| e.last_name employee_name,
       e.hire_date,
       SUM(tsr.total) total_sales
FROM total_sales_report tsr
INNER JOIN employee e ON e.employee_id = tsr.support_rep_id
GROUP BY 1;

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.510000000004
Margaret Park,2017-05-03 00:00:00,1584.0000000000032
Steve Johnson,2017-10-17 00:00:00,1393.9200000000028


Steve started six months later, so that's why his numbers are a bit behind.

# Analyzing Sales by Country

In [11]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Sales for the bottom three countries is low. It might be suitable to run small campaigns to see if it is worthwhile spending time/resources.